In [199]:
# 필요한 패키지 import
import pandas as pd
import pandasql as ps
from sklearn.preprocessing import MinMaxScaler

In [85]:
# 데이터 불러오기
data_review = pd.read_csv('DFE_project_data_raw.csv')
data_corp_info = pd.read_csv('corp_info.csv')

In [86]:
# 컬럼 정보 확인
data_review.info()
data_corp_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 563922 entries, 0 to 563921
Data columns (total 19 columns):
회사이름       563922 non-null object
회사이름_수정    563922 non-null object
회사코드       563922 non-null int64
리뷰코드       563922 non-null int64
직종구분       563922 non-null object
재직상태       563922 non-null object
근무지역       563922 non-null object
등록일자       563922 non-null object
별점평가       563922 non-null int64
승진기회       563922 non-null int64
복지급여       563922 non-null int64
워라밸        563922 non-null int64
사내문화       563922 non-null int64
경영진        563922 non-null int64
기업장점       563922 non-null object
기업단점       563920 non-null object
바라는점       563407 non-null object
성장예상       404582 non-null object
추천여부       563922 non-null object
dtypes: int64(8), object(11)
memory usage: 81.7+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102416 entries, 0 to 102415
Data columns (total 8 columns):
고객명       102414 non-null object
고객명_수정    102414 non-null object
기준년월      102416 non-nu

### 기업신용정보 데이터 중 동일한 기준년월에 대기업여부/상장여부/부도율/평가점수/평가등급이 2개 이상 존재하는 데이터가 있어 해당 이상 데이터 삭제

In [87]:

q = """

select *

  from data_corp_info

where 고객명 not in (
                    select distinct 고객명

                    from (
                            select distinct 고객명
                                , 고객명_수정
                                , 기준년월
                                , count(고객명) as cnt                        

                            from data_corp_info

                            group by 고객명
                                , 고객명_수정
                                , 기준년월
                            )
                    where cnt >= 2
                    )
    """


In [88]:
data_corp_info = ps.sqldf(q, locals())

### 기업신용정보 기업당 하나의 record만 존재하도록데이터 정리
### 201612와 201712에 데이터가 존재하는 기업만 매핑

In [89]:

q = """

with t1 as
(
    select 고객명
         , 고객명_수정
         , 대기업여부 as 대기업여부_201612
         , 상장구분 as 상장구분_201612
         , 부도율 as 부도율_201612
         , 최종점수 as 최종점수_201612
         , 최종등급 as 최종등급_201612

      from data_corp_info

     where 1=1
       and 고객명 is not null
       and 기준년월 = '201612' 
),

t2 as
(
    select 고객명
         , 고객명_수정
         , 대기업여부 as 대기업여부_201712
         , 상장구분 as 상장구분_201712
         , 부도율 as 부도율_201712
         , 최종점수 as 최종점수_201712
         , 최종등급 as 최종등급_201712

      from data_corp_info

     where 1=1
       and 고객명 is not null
       and 기준년월 = '201712' 
)

select t2.고객명
     , t2.고객명_수정
     , 대기업여부_201612
     , 상장구분_201612
     , 부도율_201612
     , 최종점수_201612
     , 최종등급_201612
     , 대기업여부_201712
     , 상장구분_201712
     , 부도율_201712
     , 최종점수_201712
     , 최종등급_201712

  from t1,
       t2
       
 where 1=1
   and t1.고객명_수정 = t2.고객명_수정

order by 2
    """


In [90]:
# sql 돌려서 기존 데이터에 덮기
data_corp_info = ps.sqldf(q, locals())

In [91]:
# sql 수행 결과 확인
data_corp_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33801 entries, 0 to 33800
Data columns (total 12 columns):
고객명             33801 non-null object
고객명_수정          33801 non-null object
대기업여부_201612    33801 non-null int64
상장구분_201612     33801 non-null int64
부도율_201612      33801 non-null float64
최종점수_201612     33801 non-null float64
최종등급_201612     33801 non-null int64
대기업여부_201712    33801 non-null int64
상장구분_201712     33801 non-null int64
부도율_201712      33801 non-null float64
최종점수_201712     33801 non-null float64
최종등급_201712     33801 non-null int64
dtypes: float64(4), int64(6), object(2)
memory usage: 3.1+ MB


In [92]:
data_corp_info.head()

,고객명,고객명_수정,대기업여부_201612,상장구분_201612,부도율_201612,최종점수_201612,최종등급_201612,대기업여부_201712,상장구분_201712,부도율_201712,최종점수_201712,최종등급_201712
0,032디자인주식회사,032디자인,2,9,1.415,55.2,7,2,9,0.840,58.5,6
1,(주)101익스피어리언스,101익스피어리언스,2,9,0.840,67.1,6,2,9,0.220,74.0,4
2,(주)153신성,153신성,2,9,0.470,75.0,5,2,9,0.470,70.7,5
3,21세기실리카(주),21세기실리카,2,9,10.038,39.7,14,2,9,10.038,38.9,14
4,(주)21세기축산,21세기축산,2,9,2.284,52.7,9,2,9,1.415,56.3,8


In [93]:
q = """

with t1 as
(
select distinct 회사이름
     , 회사이름_수정
     
     , count(리뷰코드) as 리뷰수_tot
     , sum(case when 재직상태 = '현직원' then 1 else 0 end)/count(리뷰코드) as 현직원비중_tot
     , avg(별점평가) as 별점평가_tot
     , avg(승진기회) as 승진기회_tot
     , avg(복지급여) as 복지급여_tot
     , avg(워라밸) as 워라밸_tot
     , avg(사내문화) as 사내문화_tot
     , avg(경영진) as 경영진_tot
     , sum(case when 성장예상 = '성장' then 1 else 0 end)/count(리뷰코드) as 성장긍정률_tot
     , sum(case when 추천여부 = '이 기업을 추천 합니다!' then 1 else 0 end)/count(리뷰코드) as 추천율_tot
  
  from data_review

 where 1=1
   and substr(replace(등록일자, '-',''),1,6) <= '201612'

group by 회사이름
       , 회사이름_수정
),

t2 as
(
select distinct 회사이름
     , 회사이름_수정
     
     , count(리뷰코드) as 리뷰수_m0
     , sum(case when 재직상태 = '현직원' then 1 else 0 end)/count(리뷰코드) as 현직원비중_m0
     , avg(별점평가) as 별점평가_m0
     , avg(승진기회) as 승진기회_m0
     , avg(복지급여) as 복지급여_m0
     , avg(워라밸) as 워라밸_m0
     , avg(사내문화) as 사내문화_m0
     , avg(경영진) as 경영진_m0
     , sum(case when 성장예상 = '성장' then 1 else 0 end)/count(리뷰코드) as 성장긍정률_m0
     , sum(case when 추천여부 = '이 기업을 추천 합니다!' then 1 else 0 end)/count(리뷰코드) as 추천율_m0
  
  from data_review

 where 1=1
   and substr(replace(등록일자, '-',''),1,6) = '201612'

group by 회사이름
       , 회사이름_수정
),

t3 as
(
select distinct 회사이름
     , 회사이름_수정
     
     , count(리뷰코드) as 리뷰수_m1
     , sum(case when 재직상태 = '현직원' then 1 else 0 end)/count(리뷰코드) as 현직원비중_m1
     , avg(별점평가) as 별점평가_m1
     , avg(승진기회) as 승진기회_m1
     , avg(복지급여) as 복지급여_m1
     , avg(워라밸) as 워라밸_m1
     , avg(사내문화) as 사내문화_m1
     , avg(경영진) as 경영진_m1
     , sum(case when 성장예상 = '성장' then 1 else 0 end)/count(리뷰코드) as 성장긍정률_m1
     , sum(case when 추천여부 = '이 기업을 추천 합니다!' then 1 else 0 end)/count(리뷰코드) as 추천율_m1
  
  from data_review

 where 1=1
   and substr(replace(등록일자, '-',''),1,6) between '201611' and '201612'

group by 회사이름
       , 회사이름_수정
),

t4 as
(
select distinct 회사이름
     , 회사이름_수정
     
     , count(리뷰코드) as 리뷰수_m2
     , sum(case when 재직상태 = '현직원' then 1 else 0 end)/count(리뷰코드) as 현직원비중_m2
     , avg(별점평가) as 별점평가_m2
     , avg(승진기회) as 승진기회_m2
     , avg(복지급여) as 복지급여_m2
     , avg(워라밸) as 워라밸_m2
     , avg(사내문화) as 사내문화_m2
     , avg(경영진) as 경영진_m2
     , sum(case when 성장예상 = '성장' then 1 else 0 end)/count(리뷰코드) as 성장긍정률_m2
     , sum(case when 추천여부 = '이 기업을 추천 합니다!' then 1 else 0 end)/count(리뷰코드) as 추천율_m2
  
  from data_review

 where 1=1
   and substr(replace(등록일자, '-',''),1,6) between '201610' and '201612'

group by 회사이름
       , 회사이름_수정
),

t5 as
(
select distinct 회사이름
     , 회사이름_수정
     
     , count(리뷰코드) as 리뷰수_m6
     , sum(case when 재직상태 = '현직원' then 1 else 0 end)/count(리뷰코드) as 현직원비중_m6
     , avg(별점평가) as 별점평가_m6
     , avg(승진기회) as 승진기회_m6
     , avg(복지급여) as 복지급여_m6
     , avg(워라밸) as 워라밸_m6
     , avg(사내문화) as 사내문화_m6
     , avg(경영진) as 경영진_m6
     , sum(case when 성장예상 = '성장' then 1 else 0 end)/count(리뷰코드) as 성장긍정률_m6
     , sum(case when 추천여부 = '이 기업을 추천 합니다!' then 1 else 0 end)/count(리뷰코드) as 추천율_m6
  
  from data_review

 where 1=1
   and substr(replace(등록일자, '-',''),1,6) between '201607' and '201612'

group by 회사이름
       , 회사이름_수정
)





select distinct t1.회사이름
     , t1.회사이름_수정

     , 리뷰수_tot
     , 현직원비중_tot
     , 별점평가_tot
     , 승진기회_tot
     , 복지급여_tot
     , 워라밸_tot
     , 사내문화_tot
     , 경영진_tot
     , 성장긍정률_tot
     , 추천율_tot

     , 리뷰수_m0
     , 현직원비중_m0
     , 별점평가_m0
     , 승진기회_m0
     , 복지급여_m0
     , 워라밸_m0
     , 사내문화_m0
     , 경영진_m0
     , 성장긍정률_m0
     , 추천율_m0
     
     , 리뷰수_m1
     , 현직원비중_m1
     , 별점평가_m1
     , 승진기회_m1
     , 복지급여_m1
     , 워라밸_m1
     , 사내문화_m1
     , 경영진_m1
     , 성장긍정률_m1
     , 추천율_m1

     , 리뷰수_m2
     , 현직원비중_m2
     , 별점평가_m2
     , 승진기회_m2
     , 복지급여_m2
     , 워라밸_m2
     , 사내문화_m2
     , 경영진_m2
     , 성장긍정률_m2
     , 추천율_m2

     , 리뷰수_m6
     , 현직원비중_m6
     , 별점평가_m6
     , 승진기회_m6
     , 복지급여_m6
     , 워라밸_m6
     , 사내문화_m6
     , 경영진_m6
     , 성장긍정률_m6
     , 추천율_m6

  from t1
       left outer join
       t2 on t1.회사이름 = t2.회사이름
       left outer join
       t3 on t1.회사이름 = t3.회사이름
       left outer join
       t4 on t1.회사이름 = t4.회사이름
       left outer join
       t5 on t1.회사이름 = t5.회사이름

 where 1=1

 
"""

In [94]:
data_review = ps.sqldf(q, locals())

In [124]:
data_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19999 entries, 0 to 19998
Data columns (total 52 columns):
회사이름         19999 non-null object
회사이름_수정      19999 non-null object
리뷰수_tot      19999 non-null int64
현직원비중_tot    19999 non-null int64
별점평가_tot     19999 non-null float64
승진기회_tot     19999 non-null float64
복지급여_tot     19999 non-null float64
워라밸_tot      19999 non-null float64
사내문화_tot     19999 non-null float64
경영진_tot      19999 non-null float64
성장긍정률_tot    19999 non-null int64
추천율_tot      19999 non-null int64
리뷰수_m0       3913 non-null float64
현직원비중_m0     3913 non-null float64
별점평가_m0      3913 non-null float64
승진기회_m0      3913 non-null float64
복지급여_m0      3913 non-null float64
워라밸_m0       3913 non-null float64
사내문화_m0      3913 non-null float64
경영진_m0       3913 non-null float64
성장긍정률_m0     3913 non-null float64
추천율_m0       3913 non-null float64
리뷰수_m1       6562 non-null float64
현직원비중_m1     6562 non-null float64
별점평가_m1      6562 non-null float64
승진기회_m1      6

In [96]:
data_review.head()

,회사이름,회사이름_수정,리뷰수_tot,현직원비중_tot,별점평가_tot,승진기회_tot,복지급여_tot,워라밸_tot,사내문화_tot,경영진_tot,...,리뷰수_m6,현직원비중_m6,별점평가_m6,승진기회_m6,복지급여_m6,워라밸_m6,사내문화_m6,경영진_m6,성장긍정률_m6,추천율_m6
0,라임아이(주),라임아이,6,0,33.333333,36.666667,40.000000,56.666667,36.666667,30.000000,...,4.0,0.0,35.0,35.000000,40.000000,55.000000,45.000000,30.000000,0.0,0.0
1,알파(주),알파,15,0,25.333333,29.333333,22.666667,33.333333,34.666667,34.666667,...,5.0,0.0,28.0,24.000000,20.000000,36.000000,44.000000,40.000000,0.0,0.0
2,에코마인(주),에코마인,1,0,80.000000,60.000000,60.000000,60.000000,80.000000,60.000000,...,1.0,0.0,80.0,60.000000,60.000000,60.000000,80.000000,60.000000,0.0,0.0
3,(사)국제경영원,국제경영원,10,0,56.000000,54.000000,54.000000,66.000000,60.000000,48.000000,...,3.0,0.0,60.0,53.333333,53.333333,93.333333,86.666667,66.666667,0.0,0.0
4,(사)국제교류증진협회,국제교류증진협회,5,0,52.000000,56.000000,52.000000,48.000000,44.000000,36.000000,...,2.0,0.0,50.0,60.000000,60.000000,50.000000,50.000000,40.000000,0.0,0.0


In [306]:
q = """
select t1.*
     , t2.*

  from data_review t1
       inner join
       data_corp_info t2
    on t1.회사이름_수정 = t2.고객명_수정

 where 1=1
   """

In [347]:
data = ps.sqldf(q, locals())

In [348]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4640 entries, 0 to 4639
Data columns (total 64 columns):
회사이름            4640 non-null object
회사이름_수정         4640 non-null object
리뷰수_tot         4640 non-null int64
현직원비중_tot       4640 non-null int64
별점평가_tot        4640 non-null float64
승진기회_tot        4640 non-null float64
복지급여_tot        4640 non-null float64
워라밸_tot         4640 non-null float64
사내문화_tot        4640 non-null float64
경영진_tot         4640 non-null float64
성장긍정률_tot       4640 non-null int64
추천율_tot         4640 non-null int64
리뷰수_m0          1220 non-null float64
현직원비중_m0        1220 non-null float64
별점평가_m0         1220 non-null float64
승진기회_m0         1220 non-null float64
복지급여_m0         1220 non-null float64
워라밸_m0          1220 non-null float64
사내문화_m0         1220 non-null float64
경영진_m0          1220 non-null float64
성장긍정률_m0        1220 non-null float64
추천율_m0          1220 non-null float64
리뷰수_m1          1952 non-null float64
현직원비중_m1        1952 non-null

In [349]:
data = data.filter(['대기업여부_201612','상장구분_201612','부도율_201612','최종점수_201612','최종점수_201712','리뷰수_tot',
                         '현직원비중_tot','별점평가_tot','승진기회_tot','복지급여_tot','워라밸_tot','사내문화_tot','경영진_tot',
                         '성장긍정률_tot','추천율_tot'], axis=1)

In [350]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4640 entries, 0 to 4639
Data columns (total 15 columns):
대기업여부_201612    4640 non-null int64
상장구분_201612     4640 non-null int64
부도율_201612      4640 non-null float64
최종점수_201612     4640 non-null float64
최종점수_201712     4640 non-null float64
리뷰수_tot         4640 non-null int64
현직원비중_tot       4640 non-null int64
별점평가_tot        4640 non-null float64
승진기회_tot        4640 non-null float64
복지급여_tot        4640 non-null float64
워라밸_tot         4640 non-null float64
사내문화_tot        4640 non-null float64
경영진_tot         4640 non-null float64
성장긍정률_tot       4640 non-null int64
추천율_tot         4640 non-null int64
dtypes: float64(9), int64(6)
memory usage: 543.8 KB


In [351]:
dummies1 = pd.get_dummies(data['대기업여부_201612'],prefix='대기업여부더미')
dummies2 = pd.get_dummies(data['상장구분_201612'], prefix='상장여부더미')

In [352]:
data = pd.concat([data,dummies1,dummies2], axis = 1)

In [353]:
data.head()

,대기업여부_201612,상장구분_201612,부도율_201612,최종점수_201612,최종점수_201712,리뷰수_tot,현직원비중_tot,별점평가_tot,승진기회_tot,복지급여_tot,...,성장긍정률_tot,추천율_tot,대기업여부더미_1,대기업여부더미_2,상장여부더미_1,상장여부더미_2,상장여부더미_3,상장여부더미_4,상장여부더미_5,상장여부더미_9
0,2,9,10.038,29.2,23.7,6,0,33.333333,36.666667,40.000000,...,0,0,0,1,0,0,0,0,0,1
1,2,9,2.284,48.3,33.6,15,0,25.333333,29.333333,22.666667,...,0,0,0,1,0,0,0,0,0,1
2,2,4,0.470,67.4,61.9,74,0,41.081081,56.486486,53.243243,...,0,0,0,1,0,0,0,1,0,0
3,2,9,2.284,39.3,23.6,19,0,32.631579,37.894737,25.263158,...,0,0,0,1,0,0,0,0,0,1
4,1,9,0.470,62.6,63.0,77,0,54.545455,50.649351,60.259740,...,0,0,1,0,0,0,0,0,0,1


In [354]:
data = data.iloc[:,2:23]

In [355]:
data.head()

,부도율_201612,최종점수_201612,최종점수_201712,리뷰수_tot,현직원비중_tot,별점평가_tot,승진기회_tot,복지급여_tot,워라밸_tot,사내문화_tot,...,성장긍정률_tot,추천율_tot,대기업여부더미_1,대기업여부더미_2,상장여부더미_1,상장여부더미_2,상장여부더미_3,상장여부더미_4,상장여부더미_5,상장여부더미_9
0,10.038,29.2,23.7,6,0,33.333333,36.666667,40.000000,56.666667,36.666667,...,0,0,0,1,0,0,0,0,0,1
1,2.284,48.3,33.6,15,0,25.333333,29.333333,22.666667,33.333333,34.666667,...,0,0,0,1,0,0,0,0,0,1
2,0.470,67.4,61.9,74,0,41.081081,56.486486,53.243243,25.675676,44.054054,...,0,0,0,1,0,0,0,1,0,0
3,2.284,39.3,23.6,19,0,32.631579,37.894737,25.263158,28.421053,41.052632,...,0,0,0,1,0,0,0,0,0,1
4,0.470,62.6,63.0,77,0,54.545455,50.649351,60.259740,54.285714,54.805195,...,0,0,1,0,0,0,0,0,0,1


In [356]:
scaler = MinMaxScaler() 

In [357]:
data1 = data[['부도율_201612','리뷰수_tot','별점평가_tot','승진기회_tot',
             '복지급여_tot','워라밸_tot','사내문화_tot','경영진_tot']]

In [358]:
data1.head()

,부도율_201612,리뷰수_tot,별점평가_tot,승진기회_tot,복지급여_tot,워라밸_tot,사내문화_tot,경영진_tot
0,10.038,6,33.333333,36.666667,40.000000,56.666667,36.666667,30.000000
1,2.284,15,25.333333,29.333333,22.666667,33.333333,34.666667,34.666667
2,0.470,74,41.081081,56.486486,53.243243,25.675676,44.054054,33.513514
3,2.284,19,32.631579,37.894737,25.263158,28.421053,41.052632,27.368421
4,0.470,77,54.545455,50.649351,60.259740,54.285714,54.805195,43.376623


In [359]:
scaler.fit(data1) 

C:\Users\YJH\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


MinMaxScaler(copy=True, feature_range=(0, 1))

In [361]:
data1 = pd.DataFrame(scaler.transform(data1),columns=['부도율_201612','리뷰수_tot','별점평가_tot','승진기회_tot','복지급여_tot',
                                                      '워라밸_tot','사내문화_tot','경영진_tot'])

In [362]:
data1.head()

,부도율_201612,리뷰수_tot,별점평가_tot,승진기회_tot,복지급여_tot,워라밸_tot,사내문화_tot,경영진_tot
0,0.100110,0.002078,0.166667,0.208333,0.250000,0.458333,0.208333,0.125000
1,0.022547,0.005819,0.066667,0.116667,0.033333,0.166667,0.183333,0.183333
2,0.004401,0.030341,0.263514,0.456081,0.415541,0.070946,0.300676,0.168919
3,0.022547,0.007481,0.157895,0.223684,0.065789,0.105263,0.263158,0.092105
4,0.004401,0.031588,0.431818,0.383117,0.503247,0.428571,0.435065,0.292208


In [363]:
data2 = data[['현직원비중_tot','성장긍정률_tot','추천율_tot','대기업여부더미_1','대기업여부더미_2',
              '상장여부더미_1','상장여부더미_2','상장여부더미_3','상장여부더미_4','상장여부더미_5','상장여부더미_9',
              '최종점수_201612','최종점수_201712']]

In [364]:
data_clean = pd.concat([data1,data2],axis=1)

In [368]:
data_clean.head()

,부도율_201612,리뷰수_tot,별점평가_tot,승진기회_tot,복지급여_tot,워라밸_tot,사내문화_tot,경영진_tot,현직원비중_tot,성장긍정률_tot,...,대기업여부더미_1,대기업여부더미_2,상장여부더미_1,상장여부더미_2,상장여부더미_3,상장여부더미_4,상장여부더미_5,상장여부더미_9,최종점수_201612,최종점수_201712
0,0.100110,0.002078,0.166667,0.208333,0.250000,0.458333,0.208333,0.125000,0,0,...,0,1,0,0,0,0,0,1,29.2,23.7
1,0.022547,0.005819,0.066667,0.116667,0.033333,0.166667,0.183333,0.183333,0,0,...,0,1,0,0,0,0,0,1,48.3,33.6
2,0.004401,0.030341,0.263514,0.456081,0.415541,0.070946,0.300676,0.168919,0,0,...,0,1,0,0,0,1,0,0,67.4,61.9
3,0.022547,0.007481,0.157895,0.223684,0.065789,0.105263,0.263158,0.092105,0,0,...,0,1,0,0,0,0,0,1,39.3,23.6
4,0.004401,0.031588,0.431818,0.383117,0.503247,0.428571,0.435065,0.292208,0,0,...,1,0,0,0,0,0,0,1,62.6,63.0


In [369]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4640 entries, 0 to 4639
Data columns (total 21 columns):
부도율_201612     4640 non-null float64
리뷰수_tot        4640 non-null float64
별점평가_tot       4640 non-null float64
승진기회_tot       4640 non-null float64
복지급여_tot       4640 non-null float64
워라밸_tot        4640 non-null float64
사내문화_tot       4640 non-null float64
경영진_tot        4640 non-null float64
현직원비중_tot      4640 non-null int64
성장긍정률_tot      4640 non-null int64
추천율_tot        4640 non-null int64
대기업여부더미_1      4640 non-null uint8
대기업여부더미_2      4640 non-null uint8
상장여부더미_1       4640 non-null uint8
상장여부더미_2       4640 non-null uint8
상장여부더미_3       4640 non-null uint8
상장여부더미_4       4640 non-null uint8
상장여부더미_5       4640 non-null uint8
상장여부더미_9       4640 non-null uint8
최종점수_201612    4640 non-null float64
최종점수_201712    4640 non-null float64
dtypes: float64(10), int64(3), uint8(8)
memory usage: 507.6 KB


In [366]:
from sklearn.model_selection import train_test_split

In [374]:
Y = data_clean['최종점수_201712']
X = data_clean.iloc[:,0:20]

In [376]:
X.head()

,부도율_201612,리뷰수_tot,별점평가_tot,승진기회_tot,복지급여_tot,워라밸_tot,사내문화_tot,경영진_tot,현직원비중_tot,성장긍정률_tot,추천율_tot,대기업여부더미_1,대기업여부더미_2,상장여부더미_1,상장여부더미_2,상장여부더미_3,상장여부더미_4,상장여부더미_5,상장여부더미_9,최종점수_201612
0,0.100110,0.002078,0.166667,0.208333,0.250000,0.458333,0.208333,0.125000,0,0,0,0,1,0,0,0,0,0,1,29.2
1,0.022547,0.005819,0.066667,0.116667,0.033333,0.166667,0.183333,0.183333,0,0,0,0,1,0,0,0,0,0,1,48.3
2,0.004401,0.030341,0.263514,0.456081,0.415541,0.070946,0.300676,0.168919,0,0,0,0,1,0,0,0,1,0,0,67.4
3,0.022547,0.007481,0.157895,0.223684,0.065789,0.105263,0.263158,0.092105,0,0,0,0,1,0,0,0,0,0,1,39.3
4,0.004401,0.031588,0.431818,0.383117,0.503247,0.428571,0.435065,0.292208,0,0,0,1,0,0,0,0,0,0,1,62.6


In [377]:
Y.head()

0    23.7
1    33.6
2    61.9
3    23.6
4    63.0
Name: 최종점수_201712, dtype: float64

In [379]:
X.corr()

,부도율_201612,리뷰수_tot,별점평가_tot,승진기회_tot,복지급여_tot,워라밸_tot,사내문화_tot,경영진_tot,현직원비중_tot,성장긍정률_tot,추천율_tot,대기업여부더미_1,대기업여부더미_2,상장여부더미_1,상장여부더미_2,상장여부더미_3,상장여부더미_4,상장여부더미_5,상장여부더미_9,최종점수_201612
부도율_201612,1.000000,-0.053035,-0.062003,-0.020325,-0.073580,-0.033366,-0.023970,-0.038814,0.022453,-0.009326,0.009171,-0.055860,0.055860,-0.053897,-0.030920,0.001015,-0.008750,0.007706,0.062116,-0.553281
리뷰수_tot,-0.053035,1.000000,0.110707,0.068341,0.168628,0.029541,0.070952,0.059146,-0.079660,-0.072654,-0.069945,0.273763,-0.273763,0.292738,0.035922,0.013953,-0.008863,-0.010117,-0.239865,0.127946
별점평가_tot,-0.062003,0.110707,1.000000,0.597971,0.679748,0.540925,0.655180,0.729692,0.159685,0.273395,0.345565,0.169244,-0.169244,0.132088,-0.000908,0.022230,-0.033889,0.007200,-0.093625,0.148729
승진기회_tot,-0.020325,0.068341,0.597971,1.000000,0.502264,0.249492,0.491379,0.559499,0.111549,0.271127,0.254675,0.050215,-0.050215,0.063960,-0.012766,0.019931,-0.029348,0.003090,-0.035093,0.047458
복지급여_tot,-0.073580,0.168628,0.679748,0.502264,1.000000,0.372514,0.430079,0.537639,0.057831,0.185510,0.261438,0.274460,-0.274460,0.195971,0.023857,0.027296,0.005686,-0.012633,-0.163993,0.175465
워라밸_tot,-0.033366,0.029541,0.540925,0.249492,0.372514,1.000000,0.500039,0.440356,0.097065,0.132156,0.270477,0.098066,-0.098066,0.085547,-0.025686,0.006754,-0.006933,-0.016192,-0.042386,0.123163
사내문화_tot,-0.023970,0.070952,0.655180,0.491379,0.430079,0.500039,1.000000,0.652108,0.120689,0.254593,0.297939,0.048805,-0.048805,0.044206,-0.023876,0.023694,-0.023852,0.012231,-0.015422,0.073086
경영진_tot,-0.038814,0.059146,0.729692,0.559499,0.537639,0.440356,0.652108,1.000000,0.151845,0.324634,0.373343,0.092016,-0.092016,0.081873,-0.013213,0.013181,-0.030411,0.018274,-0.049038,0.092560
현직원비중_tot,0.022453,-0.079660,0.159685,0.111549,0.057831,0.097065,0.120689,0.151845,1.000000,0.287478,0.260745,-0.082493,0.082493,-0.049339,-0.039271,-0.012233,-0.021666,0.018369,0.067260,-0.037694
성장긍정률_tot,-0.009326,-0.072654,0.273395,0.271127,0.185510,0.132156,0.254593,0.324634,0.287478,1.000000,0.484990,-0.094458,0.094458,-0.071678,-0.049957,-0.010946,-0.019385,-0.012839,0.094773,-0.003956


In [378]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

In [381]:
from sklearn.linear_model import LinearRegression

In [383]:
lr = LinearRegression()

In [384]:
lr.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [385]:
lr.score(X_train,y_train)

0.7227391582031397

In [386]:
lr.score(X_test, y_test)

0.7382852123539947